In [13]:
%load_ext dotenv
%dotenv
import openai
import re
import requests
import sys
from num2words import num2words
import os
from pprint import pprint
import pandas as pd
import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity
from transformers import GPT2TokenizerFast

API_KEY = os.environ.get("AZURE_OPENAI_API_KEY")
RESOURCE_ENDPOINT = os.environ.get("AZURE_OPENAI_ENDPOINT")

openai.api_type = "azure"
openai.api_key = API_KEY
openai.api_base = RESOURCE_ENDPOINT
openai.api_version = "2022-12-01"
print(openai.api_base)
url = openai.api_base + "/openai/deployments?api-version=2022-12-01"

r = requests.get(url, headers={"api-key": API_KEY})

print(r.text)

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv
https://amit-sandbox.openai.azure.com/
{
  "data": [
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "text-ada-001",
      "owner": "organization-owner",
      "id": "Ada1",
      "status": "succeeded",
      "created_at": 1676559783,
      "updated_at": 1676559783,
      "object": "deployment"
    },
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "text-embedding-ada-002",
      "owner": "organization-owner",
      "id": "embedding-model",
      "status": "succeeded",
      "created_at": 1676563276,
      "updated_at": 1676563276,
      "object": "deployment"
    },
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "text-davinci-003",
      "owner": "organization-owner",
      "id": "davinchi-003",
      "status": "succeeded",
      "created_at": 1676563605,
      "updated_at": 167656360

In [14]:
def normalize_text(s, sep_token = " \n "):
    print(s)
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    
    return s

In [15]:
def generateTextOpenAI(prompt):
    response = openai.Completion.create(engine="davinchi-003", prompt=prompt, max_tokens=1024)
    return response['choices'][0]['text']

In [16]:
with open("data/openning bank account.txt", "r") as file:
    doc = file.read()


In [17]:

# FAQ Example

prompt = f"""
            {normalize_text(doc)}
            divide the text into sections and put it in json format with a list of dictionaries with the following format:
            [
                {{"<section 1 title>":"<the text of section 1>"}},
                {{"<section 2 title>":"<the text of section 2>"}}
            ]
            """
print(prompt)    
text = generateTextOpenAI(prompt=prompt) 
    

export from 20/02/2023


Check In
Status: Offline
Flight Check-In
You can check into your flight multiple ways. See the options below for more information!

Multiple Ways
Online check in
Check in on our Online Check-In page up to 24 hours prior to flight departure (until 60 minutes prior to departure). You will be able to view your itinerary, purchase bags/seats and print your boarding pass. Remember bags cost more at the airport! Edited.

Mobile check in
Download our free mobile app for iOS and Android to check-in and get your mobile boarding pass with ease. If you have your mobile boarding pass in hand and are not checking a bag, or just have a carry- on, you can skip the airport line and head straight to the gate!

Airport check in
Frontier offers self-service kiosk check-in at many airports. Simply bring your confirmation code, confirmation email, credit card and ID, or passport. You will be able to quickly view your itinerary, purchase checked or carry-on bags, upgrade your seatin

In [8]:
import json
sections = json.loads(normalize_text(text))
pprint(sections)


            [
                {
                    "check in status": "Offline Flight Check-In You can check into your flight multiple ways. See the options below for more information! Multiple Ways" 
                },
                {
                    "Online Check-in": "Check in on our Online Check-In page up to 24 hours prior to flight departure (until 60 minutes prior to departure). You will be able to view your itinerary, purchase bags/seats and print your boarding pass. Remember bags cost more at the airport!"
                },
                {
                    "Mobile Check-in": "Download our free mobile app for iOS and Android to check-in and get your mobile boarding pass with ease. If you have your mobile boarding pass in hand and are not checking a bag, or just have a carry-on, you can skip the airport line and head straight to the gate!"
                },
                {
                    "Airport Check-in": "Frontier offers self-service kiosk check-in at ma

In [21]:
FAQs = []
for i in sections:
    content = list(i.values())[0]
    prompt = f"""
        {content}
        
        generate FAQ questions and answers into json text with the following format (write only the json text):
        [
            {{"<question 1>":"<answer 1>"}},
            {{"<question 2>":"<answer 2>"}}
        ]
    """
    print(prompt)
    text = generateTextOpenAI(prompt=prompt)
    try:
        FAQ_dict = json.loads(normalize_text(text))
        FAQs.append({list(i.keys())[0]: FAQ_dict})
    except Exception as e:
        print(e)
        print("Error in parsing the json text")

pprint(FAQs)


        Offline Flight Check-In You can check into your flight multiple ways. See the options below for more information! Multiple Ways
        
        generate FAQ questions and answers into json text with the following format (write only the json text):
        [
            {"<question 1>":"<answer 1>"},
            {"<question 2>":"<answer 2>"}
        ]
    

[ 
    {"What options do I have for checking in to my flight?": "You can check into your flight multiple ways, including online check-in, print-at-home check-in, kiosk and airport check-in, and mobile check-in."}, 
    {"What documents do I need to check in to my flight?": "You will need a valid form of ID (such as a passport, driver license, or government-issued ID) in order to check into your flight."} 
]

        Check in on our Online Check-In page up to 24 hours prior to flight departure (until 60 minutes prior to departure). You will be able to view your itinerary, purchase bags/seats and print your boarding pass. Rem

In [22]:
print("")
for FAQ in FAQs:
    print(list(FAQ.keys())[0])
    for i in list(FAQ.values())[0]:
        print("")
        print(f"question: {list(i.keys())[0]}")
        print(f"answer:   {list(i.values())[0]}")
        
        
    


check in status

question: What options do I have for checking in to my flight?
answer:   You can check into your flight multiple ways, including online check-in, print-at-home check-in, kiosk and airport check-in, and mobile check-in.

question: What documents do I need to check in to my flight?
answer:   You will need a valid form of ID (such as a passport, driver license, or government-issued ID) in order to check into your flight.
Online Check-in

question: How do I check in online?
answer:   Check in on our Online Check-In page up to 24 hours prior to flight departure (until 60 minutes prior to departure). You will be able to view your itinerary, purchase bags/seats and print your boarding pass.

question: Do I need to pay for bags before I arrive at the airport?
answer:   Remember bags cost more at the airport!
Mobile Check-in

question: How do I get the mobile boarding pass?
answer:   Download our free mobile app for iOS and Android to check-in and get your mobile boarding pass